In [1]:
# 库文件
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skopt import gp_minimize
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
# 设置中文字体
font = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=12)  # 替换为你的中文字体文件路径
from sklearn.preprocessing import StandardScaler


In [4]:
# 读取Excel文件
excel_path = f'C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据\\数据-时间戳.xlsx'
df_sheet = pd.read_excel(excel_path, sheet_name='Sheet4') 
df_sheet.columns


Index(['P插补', 'SI插补', '铁口1温度插补', '主参数时间戳', '富氧率', '透气性指数', 'CO', 'H2', 'CO2',
       '标准风速', '富氧流量', '冷风流量', '鼓风动能', '炉腹煤气量', '炉腹煤气指数', '理论燃烧温度', '顶压',
       '富氧压力', '冷风压力', '全压差', '热风压力', '实际风速', '热风温度', '顶温东北', '顶温西南', '顶温西北',
       '顶温东南', '阻力系数', '鼓风湿度', '设定喷煤量', '上小时喷煤量'],
      dtype='object')

In [9]:
from sklearn.feature_selection import SelectKBest, f_regression

# 假设 df_sheet 是你的数据框架
X = df_sheet.drop(['SI插补', '铁口1温度插补', '主参数时间戳', '透气性指数', 'CO', 'H2', 'CO2', '标准风速',\
       '鼓风动能', '炉腹煤气量', '炉腹煤气指数', '理论燃烧温度', '顶压',\
       '全压差','实际风速', '顶温东北', '顶温西南', '顶温西北', '顶温东南',\
       '阻力系数', '鼓风湿度', '上小时喷煤量'], axis=1)  # 输入特征

y = df_sheet['SI插补']  # 假设 'SI插补' 是目标变量

# 选择 k=4 个最好的特征
selector = SelectKBest(f_regression, k=7)
X_new = selector.fit_transform(X, y)

# 获取被选中的特征的列名
selected_features = X.columns[selector.get_support()]

# 打印被选中的特征
print("被选中的特征：", selected_features)


被选中的特征： Index(['P插补', '富氧率', '富氧流量', '冷风流量', '冷风压力', '热风温度', '设定喷煤量'], dtype='object')


In [10]:
# 计算每个参数与高炉温度的相关性
correlations = df_sheet.corr()['铁口1温度插补'].abs().sort_values(ascending=False)

# 选择最相关的前几个参数
top_features = correlations.index.tolist()
# 打印最相关的参数
print("Top features:", top_features)
# 查看具体相关性数值
print("Correlation values:")
print(correlations.head(24))


Top features: ['铁口1温度插补', 'SI插补', '冷风流量', '标准风速', '热风温度', '设定喷煤量', '上小时喷煤量', '实际风速', '热风压力', '全压差', '冷风压力', '鼓风动能', '富氧压力', '鼓风湿度', '顶温东南', '炉腹煤气量', '炉腹煤气指数', 'CO', '顶温西南', '主参数时间戳', 'P插补', '阻力系数', 'H2', '透气性指数', 'CO2', '顶温东北', '理论燃烧温度', '富氧流量', '顶温西北', '顶压', '富氧率']
Correlation values:
铁口1温度插补    1.000000
SI插补       0.371647
冷风流量       0.283416
标准风速       0.282728
热风温度       0.223820
设定喷煤量      0.215069
上小时喷煤量     0.211787
实际风速       0.199947
热风压力       0.169139
全压差        0.164020
冷风压力       0.149927
鼓风动能       0.147908
富氧压力       0.134608
鼓风湿度       0.117679
顶温东南       0.113958
炉腹煤气量      0.110920
炉腹煤气指数     0.110900
CO         0.075153
顶温西南       0.073527
主参数时间戳     0.073018
P插补        0.071704
阻力系数       0.066931
H2         0.065323
透气性指数      0.053399
Name: 铁口1温度插补, dtype: float64
